## Locality-sensitive hashing

In this specific part of the homework we are asked to minhash encode the customers from the bank dataset and then build a LSH implementation exploiting the minhash encoding itself. These are the main sub-tasks that we need to complete:
- Build the _shingle matrix_. In order to perform minhashing we need to consider each observation as a finite set with the levels of its categorical features as its elements. It goes without saying that if we have numerical features we need to discretize them. This is basically one-hot encoding (+ discretization), and in this specific context the process is called _shingling_. The resulting _shingle matrix_ is a matrix with sets/customers on the column axis and one hot encoded representation of the features on the row axis.
- Perform the minhashing. We build a series of permutations of the order of the features in the row-axis of the _shingle matrix_, and for each permutation we get for each set/customer the index of the first non-zero entry. For each set/customer we thus get a minhash signature, built from retrieving the indexes. The set of the signatures is the _minhash matrix_.
- The LSH implementation itself. The _minhash matrix_ is partitioned along the permutation axis according to a chosen number of bands; if the signature of a customer/set in a band collides with the one of another, then the two customers are placed together in one bucket. It can be shown that the probability of minhash values colliding for one permutation is exactly equal to the Jaccard similarity between the sets/customers. As we increase the bandwidth (i.e. decrease the number of bands) the match is modelled by a Bernoulli random variable with a lower and lower mean: $p = [J(A,B)]^b$, with $b$ as the bandwidth and $J(A,B)$ as the Jaccard similarity between A and B. Therefore, with a strict enough choice of the bandwidth, only **very** similar observations/sets will have one or more of their bands in the _minhash matrix_ matching.

Our implementation of the whole process heavily relies on objected-oriented programming to have a tidier approach and to keep the overall pre-processing steps (from _shingling_ to minhashing) reproducible. All the functions and the classes are inside the _lsh\_bank_ package in the GitHub repository, and we will explore them step-by-step as we call them, but we will not go in-depth, since the code is available and well-commented.

First of all, we import everything that we need from the package.

In [1]:
from lsh_bank import Shingling, min_hashing, Shuffling, lsh_buckets
import pandas as pd

First of all, we instantiate an object from the `Shingling` class; this is a class keeping as attributes the _scikit-learn_ objects used to transform the original dataset in its _shingle matrix_. This allows us to replicate the shingling process at query time, thus keeping the overall pipeline consistent.

The factory/constructor method for the `Shingling` class we are using here takes as input the table of the transactions to build the instance, taking into consideration all the necessary pre-processing steps. Specifically:

- Notice that since we have some transactions referring to the same customer, we first group by some features we consider as key (not just the customer ID, which we can see is not sufficient, but also their gender and date of birth) and perform appropriate summarization operations. For example, only the most recently recorded account balance is considered.
- A number of pre-processing steps for what concerns time data are performed, such as fixing the formats, replacing dates of birth from 1800 with missing values (then imputed with the mean); the age is computed for each customer and the hour of the transaction is extracted from the average transaction time.

As we have stressed, the discretization/the shingling itself is handled by _scikit_ objects (instances of [KBins Discretizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html) and [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)) in the `__init__` method, which also assigns the (sparse) _shingle matrix_ and the customer dataset to attributes of the object.

In [2]:
trans_table = pd.read_csv('bank_transactions.csv', index_col='TransactionID')
shingling_obj = Shingling.constructor(trans_table)

At this point, we need to perform the minhashing. First of all, we need to permute the shingle matrix, and we need to do that in a reliable way. Therefore, we define a `Shuffling` class which computes the permutations given a seed and holds the permutations as a NumPy array (i.e. the original indexes in a permuted order). The class has a `shuffle` instance method which is a generator function, returning at each iteration the array permuted through fancy indexing. We consider 400 permutations of the _shingle matrix_.

With the `min_hashing` function we then iterate over the permutations, and we use the `argmax` method of the NumPy array in order to get the first non-zero elements across the first axis, the one of the rows. The result is a row of indexes for each permutation, with the whole set of rows forming the so-called _signature matrix_.

In [3]:
# The shuffling object is initialized with the number of features, the number of permutations and a seed
shuffling_obj = Shuffling(shingling_obj.shingle_matrix.shape[0], 400, 123456)
# The min_hashing function requires the shuffling object and the shingle matrix.
# Internally, the shingle matrix is passed to the generator method from the Shuffling object, which then at each iteration returns the permuted array
signatures_array = min_hashing(shuffling_obj, shingling_obj.shingle_matrix)

At this point we just need to call the function returning the dictionary containing the buckets, `lsh_buckets`. This function receives the signature matrix and the requested number of bands, thus partitioning the matrix accordingly. Each bucket has as key the minhash signature for the specific band **together with** an index to identify the band itself. The function also deals with the cases when the number of bands is not a divisor of the number of permutations (i.e. the length of a whole minhash signature): the first $length - length\;\mathrm{mod}\; {n\_bands}$ elements are considered for the split in evenly sized bands, with the remainder (of size $length\;\mathrm{mod}\;n\_bands$) considered as a separate band.

After some trial-and-error w.r.t. the size of the buckets to understand which is the appropriate number of bands, we chose 25. The choice is also informed by some probabilistic reasoning. Probabilistically speaking, for a given Jaccard similarity between A and B, $J(A,B)$ and with the number of bands set to 25, the probability of having one or more matches (one matching band or more) is simply: $1-(1-J(A,B)^\frac{400}{25})^{25}$.

As we can see from the table below, we have a meaningful probability of having one or more bands matching only for quite high values of the Jaccard similarity.

| Jaccard Similarity | $\mathbb{P}$(#bands matching >= 1)<br/> with number of bands = 25 |
|--------------------|-------------------------------------------------------------------|
| 0.95               | ≈1                                                                |
| 0.9                | 0.994                                                             |
| 0.85               | 0.854                                                             |
| 0.8                | 0.510                                                             |
| 0.75               | 0.222                                                             |
| 0.70               | 0.080                                                             |
| 0.65               | 0.025                                                             |
| 0.6                | 0.007                                                             |
| 0.55               | 0.001                                                             |

In [7]:
buckets = lsh_buckets(minhash_signatures_dataset=signatures_array, n_bands=25)

In [13]:
# average size of a bucket
sum(map(len, buckets.values()))/len(buckets.values())

2.0493877749785514

### Query time!

Now we just have to ingest the queries and return the most similar customers for each one of them. First of all, we need to pre-process the queries, which are each converted to their minhash signature. The way the pipeline was designed until now makes every step reproducible:
- The `Shingling` object hosts the scikit objects and the logic needed to replicate for a set of queries the transformation to the _shingle matrix_. This step is automated with the `transform` method.
- The `Shuffling` object hosts the array of permutations used to build the minhash signature matrix. Therefore, we can just call the `min_hashing` function with the same `Shuffling` object but with a different _shingle matrix_, the one coming from the _shingling_ of the queries.

In [ ]:
query_array = pd.read_csv('query_users.csv')
query_array_transformed = shingling_obj.transform(query_array)
query_array_signatures = min_hashing(shuffling_obj, query_array_transformed)